In [5]:
import re

In [4]:
code = """
```cpp
#include "hybrid_lru.h"

#include <algorithm>
#include <cassert>

hybrid_lru::hybrid_lru(CACHE* cache) : hybrid_lru(cache, cache->NUM_SET, cache->NUM_WAY) {}

hybrid_lru::hybrid_lru(CACHE* cache, long sets, long ways) : replacement(cache), NUM_WAY(ways), last_used_cycles(static_cast<std::size_t>(sets * ways), 0), 
                                                             reuse_distance(static_cast<std::size_t>(sets * ways), 0), temporal_distance(static_cast<std::size_t>(sets * ways), 0),
                                                             temporal_weight(0.5), reuse_weight(0.5) {}

long hybrid_lru::find_victim(uint32_t triggering_cpu, uint64_t instr_id, long set, const champsim::cache_block* current_set, champsim::address ip,
                      champsim::address full_addr, access_type type)
{
  auto begin = std::next(std::begin(last_used_cycles), set * NUM_WAY);
  auto end = std::next(begin, NUM_WAY);

  // Calculate the reuse distance and temporal distance for each block in the set
  for (auto it = begin; it != end; ++it) {
    auto way = std::distance(begin, it);
    long reuse_dist = calculate_reuse_distance(set, way);
    long temporal_dist = calculate_temporal_distance(set, way);

    // Update the reuse distance and temporal distance for this block
    reuse_distance.at((std::size_t)(set * NUM_WAY + way)) = reuse_dist;
    temporal_distance.at((std::size_t)(set * NUM_WAY + way)) = temporal_dist;
  }

  // Find the way with the minimum weighted sum of reuse distance and temporal distance
  auto victim = std::min_element(begin, end, [this, set](const auto& a, const auto& b) {
    auto way_a = std::distance(std::next(std::begin(last_used_cycles), set * NUM_WAY), &a);
    auto way_b = std::distance(std::next(std::begin(last_used_cycles), set * NUM_WAY), &b);
    long reuse_dist_a = reuse_distance.at((std::size_t)(set * NUM_WAY + way_a));
    long reuse_dist_b = reuse_distance.at((std::size_t)(set * NUM_WAY + way_b));
    long temporal_dist_a = temporal_distance.at((std::size_t)(set * NUM_WAY + way_a));
    long temporal_dist_b = temporal_distance.at((std::size_t)(set * NUM_WAY + way_b));

    long weight_sum_a = temporal_weight * temporal_dist_a + reuse_weight * reuse_dist_a;
    long weight_sum_b = temporal_weight * temporal_dist_b + reuse_weight * reuse_dist_b;

    return weight_sum_a > weight_sum_b;
  });

  assert(begin <= victim);
  assert(victim < end);
  return std::distance(begin, victim);
}

void hybrid_lru::replacement_cache_fill(uint32_t triggering_cpu, long set, long way, champsim::address full_addr, champsim::address ip, champsim::address victim_addr,
                                 access_type type)
{
  // Mark the way as being used on the current cycle
  last_used_cycles.at((std::size_t)(set * NUM_WAY + way)) = cycle++;

  // Update the reuse distance and temporal distance for this block
  long reuse_dist = calculate_reuse_distance(set, way);
  long temporal_dist = calculate_temporal_distance(set, way);

  reuse_distance.at((std::size_t)(set * NUM_WAY + way)) = reuse_dist;
  temporal_distance.at((std::size_t)(set * NUM_WAY + way)) = temporal_dist;
}

void hybrid_lru::update_replacement_state(uint32_t triggering_cpu, long set, long way, champsim::address full_addr, champsim::address ip,
                                   champsim::address victim_addr, access_type type, uint8_t hit)
{
  // Mark the way as being used on the current cycle
  if (hit && access_type{type} != access_type::WRITE) // Skip this for writeback hits
    last_used_cycles.at((std::size_t)(set * NUM_WAY + way)) = cycle++;

  // Update the reuse distance and temporal distance for this block
  long reuse_dist = calculate_reuse_distance(set, way);
  long temporal_dist = calculate_temporal_distance(set, way);

  reuse_distance.at((std::size_t)(set * NUM_WAY + way)) = reuse_dist;
  temporal_distance.at((std::size_t)(set * NUM_WAY + way)) = temporal_dist;
}

long hybrid_lru::calculate_reuse_distance(long set, long way) {
  // Calculate the reuse distance based on the time since the last access
  // For simplicity, we assume the reuse distance is the number of cycles since the last access
  return cycle - last_used_cycles.at((std::size_t)(set * NUM_WAY + way));
}

long hybrid_lru::calculate_temporal_distance(long set, long way) {
  // Calculate the temporal distance based on the time since the last access
  // For simplicity, we assume the temporal distance is the number of cycles since the last access
  return cycle - last_used_cycles.at((std::size_t)(set * NUM_WAY + way));
}
```
"""

In [6]:

match = re.search(r'(\w+)::\1\s*\(', code)

In [8]:
match.group(1)

'hybrid_lru'